## Tarea 2 del curso CM-072

* Nombre y apellidos: Bitzer Arotoma Bacilio
* Fecha de presentación: 19 de setiembre

Esta tarea utiliza un pequeño subconjunto de los datos de la competencia de Kaggle  [Yelp Business Rating Prediction ](https://www.kaggle.com/c/yelp-recsys-2013).

* Descripción de la data
    - `yelp.json` es el formato original del archivo y `yelp.csv` contiene los mismos datos, en un formato más conveniente. Ambos archivos están en esta carpeta de la tarea, por lo que no es necesario descargar los datos del sitio web de Kaggle.

    - Cada observación en este conjunto de datos es una revisión de un negocio en particular por un usuario en particular.

    - La columna "stars" es la cantidad de estrellas (1 a 5) asignadas por el revisor al negocio. (Estrellas más altas indican mejores puntuaciones.) En otras palabras, es la calificación del negocio por la persona que escribió la revisión.
 
    - La columna "cool" es la cantidad de votos "cool" que recibió esta crítica de otros usuarios de Yelp. Todas las reseñas comienzan con 0 votos "cools" y no hay límite para la cantidad de votos "cools" que puede recibir una revisión. En otras palabras, es una calificación de la revisión en sí misma, no una calificación de la empresa.
   
    - Las columnas "useful" y "funny" son similares a la columna "cool".
    
* Sugerencia: En estos ejercicios debes utilizar los siguientes commandos de scikit learn:

    - [from sklearn.linear_model import LinearRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html).
    - [from sklearn.cross_validation import train_test_split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)
    - [from sklearn import metrics](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics).

## Preguntas

1. Lea yelp.csv en un DataFrame.


In [1]:
# Tu solucion
import pandas as pd
import numpy as np
df = pd.read_csv('yelp.csv')


In [2]:
df.head()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0


2.Ignoramos el archivo `yelp.csv` y construye un DataFrame de `yelp.json`. Esto implica leer los datos en Python, decodificar el JSON, convertirlo en un DataFrame y agregar columnas individuales para cada uno de los tipos de votos.

In [ ]:
#Solución
import json 
data = []
with open('yelp.json') as f:
    for line in f:
        data.append(json.loads(line))
df = pd.DataFrame.from_dict(data)
df.head()
df.shape

(10000, 8)

In [ ]:
votes=pd.DataFrame(df['votes'].to_dict()).T
votes.head()

In [ ]:
df = df.join(pd.DataFrame(df['votes'].to_dict()).T)
df.head()

In [ ]:
df.drop(['votes'], inplace=True, axis=1)
df.head()

3.Explora la relación entre cada uno de los tipos de votos (cool/useful/funny) y la cantidad de estrellas.

In [ ]:
# Tu solucion
df_solution = votes.join(df['stars'])
df_solution.head()

In [ ]:
df_solution.corr()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
pd.scatter_matrix(df_solution, figsize=(6, 6))
plt.show()

4.Define `cool/ useful/funny` como características y `stars` como respuesta.

In [ ]:
# Tu solucion

X = df_solution.loc[:, 'cool':'useful']
#X = X.as_matrix()
y = df_solution.loc[:,'stars']
print(X.head())
print(y.head())

5.Ajustamos un modelo de regresión lineal e interpreta los coeficientes. ¿Los coeficientes tienen sentido intuitivo para ti?. Explora el sitio web de Yelp para ver si se puede detectar tendencias similares.
    

In [ ]:
# Tu solucion
from sklearn.linear_model import LinearRegression
X = X.as_matrix()
y = y.as_matrix()
lr = LinearRegression(normalize=True)

In [ ]:
lr.fit(X,y)
print('y = ' + str(lr.intercept_) + ' ')
for i, c in enumerate(lr.coef_):
    print(str(c) + ' * x' + str(i))

Para la variable x0 que representa a "cool" tiene una constante de crecimiento igual a 0.27435946858853066, para x1  que es "funny" es -0.1356744905370616 y para x2 que es "useful" es -0.14745239099401516.

6.Evalua el modelo dividiendo los datos en  conjuntos de entrenamiento y prueba y calculando el RMSE. ¿El RMSE tiene un sentido intuitivo para ti?
    

In [ ]:
# Tu solucion
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2)
lr.fit(X_train, Y_train)
#print("Score",lr.score(X_test, Y_test))
Y_pred = lr.predict(X_train)
rmse = np.sqrt(mean_squared_error(Y_train, Y_pred))
print("RMSE : ",rmse)
#r2 = lr.score(X_train, Y_train)
#print(r2)
#r3 = lr.score(X_test, Y_test)
#print(r3)


Calcular RMSE 

7.Intenta eliminar algunas de las características y analizar  si el RMSE mejora.
   

In [ ]:
# Eliminando la primera columna "cool"
filtro = np.array([False, True, True])
X1 = X_train[:,filtro]
lr.fit(X1, Y_train)
Y_pred1 = lr.predict(X1)
rmse = np.sqrt(mean_squared_error(Y_train, Y_pred1))
print("RMSE : ",rmse)
print("Score : ",lr.score(X1,Y_train))

In [ ]:
# Eliminando la segunda columna "funny"
filtro = np.array([True, False, True])
X2 = X_train[:,filtro]
lr.fit(X2, Y_train)
Y_pred2 = lr.predict(X2)
rmse = np.sqrt(mean_squared_error(Y_train, Y_pred2))
print("RMSE : ",rmse)
print("Score : ",lr.score(X2,Y_train))

In [ ]:
# Eliminando la tercera columna "useful"
filtro = np.array([True, True, False])
X3 = X_train[:,filtro]
lr.fit(X3, Y_train)
Y_pred3 = lr.predict(X3)
rmse = np.sqrt(mean_squared_error(Y_train, Y_pred3))
print("RMSE : ",rmse)
print("Score : ",lr.score(X3,Y_train))

Observando los tres casos anteriores, en la que se elimina solo un caracteristica, notamos que elRMSE en el mejor de los casos es de 1.199579958055843 que es en el caso que se elimina la caracteristica "funny".

8.(Obligatorio) Piensa en algunas características nuevas que podrías crear a partir de los datos existentes que pueden ser predictivos de la respuesta. Descubre cómo crear esas características en Pandas, agrégalas a tu modelo y ve si el RMSE mejora.
    

In [ ]:
# Tu solucion


9.Compara tu mejor RMSE en el conjunto de prueba con el RMSE para  un "modelo nulo", que es el modelo que ignora todas las características y simplemente predice el valor medio de respuesta en el conjunto de prueba.
    

In [ ]:
# Tu solucion

10.En lugar de tratar este problema por regresión, como sería si fuese un problema de clasificación y qué precisión en el conjunto prueba se puede lograr con KNN.
   

In [ ]:
# Tu solucion
from sklearn.neighbors import KNeighborsClassifier
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2)
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, Y_train)
print("Score : ",knn.score(X_test, Y_test))


11.Descubre cómo usar la regresión lineal en  clasificación y compara el valor de  precisión de clasificación con la precisión del algoritmo KNN.

In [ ]:
# Tu solucion

12.Lee el archivo `advertising.csv` y almacena este conjunto de datos en un Dataframe  llamado `datos`.

In [ ]:
# Tu solucion
datos = pd.read_csv('advertising.csv')
type(datos)

In [ ]:
datos.head()

13.Usa `info` y  `describe` en `datos`. Analiza tu respuesta.

In [ ]:
# Tu solucion
datos.info()

In [ ]:
datos.describe()


14.Usemos seaborn para explorar los datos. Realiza los siguientes gráficos  que se muestran a continuación!

   - Crea un histograma para `age`
   - Crea un `joinplot` que muestre `Area Income` versus `Age`.
   - Crea un `joinplot` que muestre las distribuciones kde de  `Daily Time spent on site` vs. `Age`.
   - Crea un `joinplot `de `Daily Time Spent on Site` vs. `Daily Internet Usage`.
   - Crea un `pairplot` con `hue` definido por la característica de la columna `Clicked on Ad`. 
   
 Analiza tus resultados.

In [ ]:
# Tu solucion
import seaborn as sb
sb.distplot(datos.Age)

In [ ]:
sb.jointplot(datos.loc[:,"Area Income"], datos.Age)


In [ ]:
sb.jointplot(datos.loc[:, "Daily Time Spent on Site"], datos.Age, kind="kde")


In [ ]:
sb.jointplot(datos.loc[:,"Daily Time Spent on Site"], datos.loc[:, "Daily Internet Usage"])


In [ ]:
sb.pairplot(datos, hue="Clicked on Ad")


15.(Obligatorio).  Asumiendo que:

```
datos.drop(['Ad Topic Line', 'City', 'Country', 'Timestamp'], axis=1, inplace=True)
X = datos.drop(['Clicked on Ad'], axis = 1)
y = datos['Clicked on Ad']
```

- Divide los datos en el conjunto de entrenamiento y prueba usando `train_test_split`.
- Entrena y ajusta un modelo de regresión logística en el conjunto de entrenamiento.
- Pronostica valores para los datos de prueba.
- Crea un informe de clasificación para el modelo. Utiliza `classification_report`.

In [ ]:
# Tu solucion
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
logR = LogisticRegression()
datos.drop(['Ad Topic Line', 'City', 'Country', 'Timestamp'], axis=1, inplace=True)
X = datos.drop(['Clicked on Ad'], axis = 1)
y = datos['Clicked on Ad']
X_train, X_test, Y_train, Y_test = train_test_split(X, y)
logR.fit(X_train, Y_train)
Y_pred = logR.predict(X_test)
print(classification_report(Y_test, Y_pred))


16.(Obligatorio) Presenta un reporte del siguiente artículo de Sebastian Raschka: [Naive Bayes and Text Classification](http://sebastianraschka.com/Articles/2014_naive_bayes_1.html).

In [ ]:
# Tu solucion